In [0]:
%%shell
%tensorflow_version 1.x
sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
pip install --user Cython
pip install --user contextlib2
pip install --user pillow
pip install --user lxml
pip install --user matplotlib


/bin/bash: line 0: fg: no job control
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://arc

In [0]:
#Import Libraries that may be required in this project

from __future__ import print_function

import collections
import io
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython import display
from sklearn import metrics



In [0]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Changing the directory to particular folder
#Make sure to have this folder
%cd '/content/drive/My Drive/ForColab'

/content/drive/My Drive/ForColab


In [0]:
%cd '/content/drive/My Drive/ForColab/Exported/Pothole-Pascal-PascalVOC-export/Annotations'

/content/drive/My Drive/ForColab/Exported/Pothole-Pascal-PascalVOC-export/Annotations


In [0]:
# For Creating Annotations From XML
# XML file was generated from VoTT
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_txt(here):
    #cls=open('/content/drive/My Drive/DAEN/class.data')
    xml_list = ''
    tree = ET.parse(here)
    root = tree.getroot()
    
    for member in root.findall('object'):
        bbx = member.find('bndbox')
        xmin = float(bbx.find('xmin').text)
        ymin = float(bbx.find('ymin').text)
        xmax = float(bbx.find('xmax').text)
        ymax = float(bbx.find('ymax').text)
        label = member.find('name').text
        
        dw=float(1/(int(root.find('size')[0].text)))
        dh=float(1/(int(root.find('size')[1].text)))

        x=float((((xmin+xmax)/2)-1)*dw)
        y=float((((ymin+ymax)/2)-1)*dh)

        h=float((ymax-ymin)*dh)
        w=float((xmax-xmin)*dw)
        
        filename=root.find('filename').text

#        xml_list=xml_list + str(0)+','+str(x)+','+str(y)+','+str(w)+','+str(h)+'\n'

        xml_list=xml_list + str(0)+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)+'\n'

    #column_name = ['class', 'x_center', 'y_center', 'width', 'height']
    f=open('/content/drive/My Drive/FinalColab/XMLAnnotations/'+filename.rsplit(".")[0]+".txt",'w')
    f.write(xml_list)
    f.close()
    
def main():
    file_path="/content/drive/My Drive/ForColab/Exported/Pothole-Pascal-PascalVOC-export/Annotations"
    read_files=glob.glob(os.path.join(file_path,"*.xml"))
    length=len(os.listdir(file_path))
    for i in read_files:
      xml_df = xml_to_txt(i)
      
main()

In [0]:
# For train file
import pandas as pd
dataframe = pd.read_csv("/content/drive/My Drive/ForColab/Paths/Pothole_train.txt", header = None)
dataframe

,0
0,38.jpg 1
1,37.jpg 1
2,36.jpg 1
3,35.jpg 1
4,34.jpg 1
...,...
227,102.jpg 1
228,101.jpg 1
229,100.jpg 1
230,10.jpg 1


In [0]:
# Creating for Train.txt
dataframe = dataframe[0].str.split(" ", n = 1, expand = True) 
dataframe = dataframe.drop(columns=1, axis = 1)
dataframe['value'] = '/content/Potholes/FinalColab/images/'
dataframe["final"] = dataframe["value"] + dataframe[0]
dataframe = dataframe.drop(columns=0, axis = 1)
dataframe = dataframe.drop(columns="value", axis = 1)
dataframe

,final
0,/content/Potholes/FinalColab/images/38.jpg
1,/content/Potholes/FinalColab/images/37.jpg
2,/content/Potholes/FinalColab/images/36.jpg
3,/content/Potholes/FinalColab/images/35.jpg
4,/content/Potholes/FinalColab/images/34.jpg
...,...
227,/content/Potholes/FinalColab/images/102.jpg
228,/content/Potholes/FinalColab/images/101.jpg
229,/content/Potholes/FinalColab/images/100.jpg
230,/content/Potholes/FinalColab/images/10.jpg


In [0]:
%cd '/content/drive/My Drive/FinalColab'

/content/drive/My Drive/FinalColab


In [0]:
# Create a new train.txt file in FinalColab folder
a = open('train.txt', 'w')
for files in dataframe['final']:
  a.write(files+'\n')
a.close()

In [0]:
# For test file
import pandas as pd
dataframe = pd.read_csv("/content/drive/My Drive/ForColab/Paths/Pothole_val.txt", header = None)
dataframe

,0
0,97.jpg 1
1,96.jpg 1
2,95.jpg 1
3,94.jpg 1
4,93.jpg 1
5,92.jpg 1
6,91.jpg 1
7,90.jpg 1
8,9.jpg 1
9,89.jpg 1


In [0]:
# Creating for test.txt
dataframe = dataframe[0].str.split(" ", n = 1, expand = True) 
dataframe = dataframe.drop(columns=1, axis = 1)
dataframe['value'] = '/content/Potholes/FinalColab/images/'
dataframe["final"] = dataframe["value"] + dataframe[0]
dataframe = dataframe.drop(columns=0, axis = 1)
dataframe = dataframe.drop(columns="value", axis = 1)
dataframe

,final
0,/content/Potholes/FinalColab/images/97.jpg
1,/content/Potholes/FinalColab/images/96.jpg
2,/content/Potholes/FinalColab/images/95.jpg
3,/content/Potholes/FinalColab/images/94.jpg
4,/content/Potholes/FinalColab/images/93.jpg
5,/content/Potholes/FinalColab/images/92.jpg
6,/content/Potholes/FinalColab/images/91.jpg
7,/content/Potholes/FinalColab/images/90.jpg
8,/content/Potholes/FinalColab/images/9.jpg
9,/content/Potholes/FinalColab/images/89.jpg


In [0]:
%cd '/content/drive/My Drive/FinalColab'

/content/drive/My Drive/FinalColab


In [0]:
# Create a new test.txt file in FinalColab folder
a = open('test.txt', 'w')
for files in dataframe['final']:
  a.write(files+'\n')
a.close()

In [0]:
# Training and Testing the model STARTS HERE
# For pothole folder creation in colab
# Should have Testing folder in My Drive
# This code is not important and was only used while checking the codes in between
%cd '/content/drive/My Drive'
%cp -av Testing/ /content/Potholes
#/content/drive/My Drive/Testing/Forfolder.txt

/content/drive/My Drive
'Testing/' -> '/content/Potholes'
'Testing/testing.txt' -> '/content/Potholes/testing.txt'
'Testing/Forfolder.txt' -> '/content/Potholes/Forfolder.txt'


In [0]:
# Importing Folder from Drive to Colab
# Folder contains all the data including images and the files that were saved earlier in this folder
%cd '/content/drive/My Drive'
%cp -av FinalColab/ /content/Potholes/FinalColab

/content/drive/My Drive
'FinalColab/' -> '/content/Potholes/FinalColab'
'FinalColab/images' -> '/content/Potholes/FinalColab/images'
'FinalColab/images/1.jpg' -> '/content/Potholes/FinalColab/images/1.jpg'
'FinalColab/images/100.jpg' -> '/content/Potholes/FinalColab/images/100.jpg'
'FinalColab/images/103.jpg' -> '/content/Potholes/FinalColab/images/103.jpg'
'FinalColab/images/106.jpg' -> '/content/Potholes/FinalColab/images/106.jpg'
'FinalColab/images/102.jpg' -> '/content/Potholes/FinalColab/images/102.jpg'
'FinalColab/images/104.jpg' -> '/content/Potholes/FinalColab/images/104.jpg'
'FinalColab/images/10.jpg' -> '/content/Potholes/FinalColab/images/10.jpg'
'FinalColab/images/108.jpg' -> '/content/Potholes/FinalColab/images/108.jpg'
'FinalColab/images/109.jpg' -> '/content/Potholes/FinalColab/images/109.jpg'
'FinalColab/images/107.jpg' -> '/content/Potholes/FinalColab/images/107.jpg'
'FinalColab/images/11.jpg' -> '/content/Potholes/FinalColab/images/11.jpg'
'FinalColab/images/105.jpg' 

In [0]:
# Change the directory for darknet repository
cd /content/

/content


In [0]:
# clone darknet repo

# Any one of the repo can be used
# First one is more commonly used, but in this project second repo was used

#! git clone https://github.com/pjreddie/darknet

! git clone https://github.com/ansarisam/darknet.git

!/usr/local/cuda/bin/nvcc --version

Cloning into 'darknet'...
remote: Enumerating objects: 5912, done.
remote: Total 5912 (delta 0), reused 0 (delta 0), pack-reused 5912
Receiving objects: 100% (5912/5912), 6.17 MiB | 4.11 MiB/s, done.
Resolving deltas: 100% (3925/3925), done.
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
#%%shell
#rm -rf darknet

In [0]:
# Automating the copying of Makefile
# Some changes are required in Make file, the updated one must be used
%cd '/content/drive/My Drive'
%cp -av FinalColab/Makefile /content/darknet

/content/drive/My Drive
'FinalColab/Makefile' -> '/content/darknet/Makefile'


In [0]:
# Automating the copying of cfg file
# Some changes were made in the original and then was copied back
%cd '/content/drive/My Drive'
%cp -av FinalColab/yolov3-voc.cfg /content/darknet/cfg

/content/drive/My Drive
'FinalColab/yolov3-voc.cfg' -> '/content/darknet/cfg/yolov3-voc.cfg'


In [0]:
# Automating the copying of cfg file TEST
# From yolov3-voc.cfg copy was made for test runs
%cd '/content/drive/My Drive'
%cp -av FinalColab/yolov3-voctest.cfg /content/darknet/cfg

/content/drive/My Drive
'FinalColab/yolov3-voctest.cfg' -> '/content/darknet/cfg/yolov3-voctest.cfg'


In [0]:
# Automating the copying of detector.c
# Made some changes in detector.c file (specifically for saving the weights)
%cd '/content/drive/My Drive'
%cp -av FinalColab/detector.c /content/darknet/examples/

/content/drive/My Drive
'FinalColab/detector.c' -> '/content/darknet/examples/detector.c'


In [0]:
# Change the working directory
cd /content/

/content


In [0]:
# before running this make changes in makefile
%%shell
cd darknet
make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:957:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result 

In [0]:
# Few directories created
# calling pretrained models

%%shell
mkdir backup
mkdir pretrained
cd pretrained
wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-04-18 07:17:58--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.06MB/s    in 2m 27s  

2020-04-18 07:20:26 (1.05 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [0]:
# To check GPU
#import tensorflow as tf
#tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# Automating the copying of input.cfg
%cd '/content/drive/My Drive'
%cp -av FinalColab/input.cfg /content/Potholes/

/content/drive/My Drive
'FinalColab/input.cfg' -> '/content/Potholes/input.cfg'


In [0]:
# Automating the copying of Pothole.names
# Pothole.names is similar to class.data, anyone can be used
%cd '/content/drive/My Drive'
%cp -av FinalColab/Pothole.names /content/Potholes/

/content/drive/My Drive
'FinalColab/Pothole.names' -> '/content/Potholes/Pothole.names'


In [0]:
# Automating the copying of train.txt
%cd '/content/Potholes/'
%cp -av FinalColab/train.txt /content/Potholes/FinalColab/images/

/content/Potholes
'FinalColab/train.txt' -> '/content/Potholes/FinalColab/images/train.txt'


In [0]:
# Automating the copying of test.txt
%cd '/content/Potholes/'
%cp -av FinalColab/test.txt /content/Potholes/FinalColab/images/

/content/Potholes
'FinalColab/test.txt' -> '/content/Potholes/FinalColab/images/test.txt'


In [0]:
# Automating the copying of class.data
%cd '/content/Potholes/'
%cp -av FinalColab/class.data /content/Potholes/FinalColab/images/

/content/Potholes
'FinalColab/class.data' -> '/content/Potholes/FinalColab/images/class.data'


In [0]:
# Automating the copying of Weights
# This command will work once you have started training and have weights saved in drive
# Ignore this command if you have not started the training and don't have weights saved
%cd '/content/drive/My Drive'
%cp -av Weights/yolov3-voc_final.weights /content/backup/

/content/drive/My Drive
'Weights/yolov3-voc_final.weights' -> '/content/backup/yolov3-voc_final.weights'


In [0]:
# Automating the copying of train.list and test.list
#%cd '/content/Potholes/'
#%cp -av FinalColab/train.list /content/darknet/data/
#%cp -av FinalColab/test.list /content/darknet/data/

In [0]:
# If you created input.cfg file in windows OS you may have to convert it into Unix format
%%shell
sudo apt install dos2unix
dos2unix /content/Potholes/input.cfg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 2s (183 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 144568 files and directories curren

In [0]:
# Changes the working directory
cd /content/

/content


In [0]:
#Original
# Run this command to start training your model
%%shell
cd darknet/
./darknet detector train /content/Potholes/input.cfg /content/darknet/cfg/yolov3-voc.cfg /content/pretrained/darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Region 106 Avg IOU: 0.653641, Class: 0.980984, Obj: 0.583987, No Obj: 0.000290, .5R: 1.000000, .75R: 0.000000,  count: 2
Region 82 Avg IOU: 0.589633, Class: 0.999292, Obj: 0.502655, No Obj: 0.003215, .5R: 1.000000, .75R: 0.000000,  count: 2
Region 94 Avg IOU: 0.562701, Class: 0.981575, Obj: 0.328143, No Obj: 0.000666, .5R: 0.750000, .75R: 0.000000,  count: 4
Region 106 Avg IOU: 0.694245, Class: 0.955237, Obj: 0.677733, No Obj: 0.000356, .5R: 1.000000, .75R: 0.333333,  count: 3
Region 82 Avg IOU: 0.518649, Class: 0.999399, Obj: 0.718348, No Obj: 0.003603, .5R: 0.500000, .75R: 0.000000,  count: 2
Region 94 Avg IOU: 0.611148, Class: 0.997104, Obj: 0.361667, No Obj: 0.000800, .5R: 0.875000, .75R: 0.000000,  count: 8
Region 106 Avg IOU: 0.615190, Class: 0.963043, Obj: 0.181733, No Obj: 0.000360, .5R: 0.800000, .75R: 0.200000,  count: 15
Region 82 Avg IOU: 0.727259, Class: 0.996231, Obj: 0.756776, No Obj: 0.004629, .5R: 1.000000, .75R: 0.250

CalledProcessError: ignored

In [0]:
# Training From Weights
# Once you have saved weights after some iterations use them and don't start training from the scratch
# Last part of the last line will be changed everytime for the new weights
%%shell
cd darknet/
./darknet detector train /content/Potholes/input.cfg /content/darknet/cfg/yolov3-voc.cfg /content/backup/yolov3-voc_4000.weights

Streaming output truncated to the last 5000 lines.
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.860812, Class: 0.999851, Obj: 0.999735, No Obj: 0.005607, .5R: 1.000000, .75R: 0.857143,  count: 7
Region 94 Avg IOU: 0.815333, Class: 0.999743, Obj: 0.997694, No Obj: 0.000663, .5R: 1.000000, .75R: 0.750000,  count: 4
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.793191, Class: 0.999969, Obj: 0.999988, No Obj: 0.002336, .5R: 1.000000, .75R: 0.666667,  count: 3
Region 94 Avg IOU: 0.845261, Class: 0.999899, Obj: 0.999768, No Obj: 0.000490, .5R: 1.000000, .75R: 1.000000,  count: 4
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000014, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.880817, Class: 0.999832, Obj: 0.999258, No Obj: 0.004164, .5R: 1.000000, .75R: 1.000000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -na

In [0]:
# Testing From Weights 
%%shell
cd darknet/
./darknet detector test /content/Potholes/input.cfg /content/darknet/cfg/yolov3-voctest.cfg /content/backup/yolov3-voc_final.weights

Couldn't open file: /content/Potholes/input.cfg


In [0]:
# Below this part (TESTING)

In [0]:
# Automating the copying of Python file
# Codes are in repo
# I do not own the python code used below, taken from another source
%cd '/content/drive/My Drive'
%cp -av Python_Code/pothhole_json.py /content/darknet
%cp -av Python_Code/bounding_box.py /content/darknet

/content/drive/My Drive
'Python_Code/pothhole_json.py' -> '/content/darknet/pothhole_json.py'
'Python_Code/bounding_box.py' -> '/content/darknet/bounding_box.py'


In [0]:
# creating folders
%%shell
cd /content/
#mkdir input_images # for images that needs to be input
mkdir output  # for the json results
mkdir predictions  # for the saving the predicted images
mkdir video_predictions #for video predictions

mkdir: cannot create directory ‘output’: File exists
mkdir: cannot create directory ‘predictions’: File exists
mkdir: cannot create directory ‘video_predictions’: File exists


CalledProcessError: ignored

In [0]:
# Automating the copying input test images
# in input_images folder, put random images for testing
# these are not the test images that were discussed earlier
%cd '/content/drive/My Drive'
%cp -av input_images/ /content/

/content/drive/My Drive
'input_images/' -> '/content/input_images'
'input_images/1pothole.jpg' -> '/content/input_images/1pothole.jpg'
'input_images/pot4.jpg' -> '/content/input_images/pot4.jpg'
'input_images/rd1.jpg' -> '/content/input_images/rd1.jpg'
'input_images/pot3.jpg' -> '/content/input_images/pot3.jpg'
'input_images/pot2.jpg' -> '/content/input_images/pot2.jpg'


In [0]:
# Creating input.txt
%cd /content/
import os

a = open('/content/input_test_images.txt', 'w')

directory='/content/input_images'

for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
      a.writelines(os.path.join(directory, filename))
      a.writelines('\n')
      print(os.path.join(directory, filename))
      continue
    else:
      continue

a.close()

/content
/content/input_images/rd1.jpg
/content/input_images/1pothole.jpg
/content/input_images/pot2.jpg
/content/input_images/pot4.jpg
/content/input_images/pot3.jpg


In [0]:
# code to run python file if it is saved
%%shell
cd /content/darknet/
python pothhole_json.py

In [0]:
%cd /content/

/content


In [0]:
# code to run python file to create bounding boxes
%%shell
cd /content/darknet/
python bounding_box.py

In [0]:
# Working on the directory for video predictions
%cp -av /content/Potholes/FinalColab/class.data /content/video_predictions # copying class.data file
%cp -av /content/Potholes/FinalColab/yolov3-voctest.cfg /content/video_predictions # copying test cfg file
%cp -av /content/backup/yolov3-voc_final.weights /content/video_predictions # copying final weights


'/content/Potholes/FinalColab/class.data' -> '/content/video_predictions/class.data'
'/content/Potholes/FinalColab/yolov3-voctest.cfg' -> '/content/video_predictions/yolov3-voctest.cfg'
'/content/backup/yolov3-voc_final.weights' -> '/content/video_predictions/yolov3-voc_final.weights'


In [0]:
# Copying Video Python Script
%cd '/content/drive/My Drive/'
%cp -av Python_Code/yolo_video.py /content/video_predictions

/content/drive/My Drive
'Python_Code/yolo_video.py' -> '/content/video_predictions/yolo_video.py'


In [0]:
# Executing the python file for video
%%shell
cd /content/video_predictions/
python yolo_video.py --input /content/video.mp4 --output /content/output.avi --yolo /content/video_predictions/


[INFO] loading YOLO from disk...
[INFO] 340 total frames in video
[INFO] single frame took 1.9322 seconds
[INFO] estimated total time to finish: 656.9501
[INFO] cleaning up...
